In [41]:
import pandas as pd

Load everything we need first.

Reminder: Download the latest 'data' folder from Teams and replace yours before start working.

In [42]:
df_influent = pd.read_parquet('data/raw-data/Influent_2023.parquet')
df_oxygen_A = pd.read_parquet('data/raw-data/oxygen_a_2023.parquet')
df_oxygen_B = pd.read_parquet('data/raw-data/oxygen_b_2023.parquet')
df_ammonium = pd.read_parquet('data/raw-data/ammonium_2023.parquet')
df_nitrite = pd.read_parquet('data/raw-data/nitrate_2023.parquet')
df_phosphate = pd.read_parquet('data/raw-data/phosphate_2023.parquet')
df_effluent = pd.read_excel('data/raw-data/effluent_2023.xlsx')
df_weather = pd.read_csv('data/weather-data/weather_2023.csv', sep=',')

Check the year of these dataframes are 2023, and see their columns names & records frenquency.

In [43]:
# influent data - recorded every minute
# df_influent.head()

# oxygen data (A & B) - need to be merged (take the average) first - recorded every minute
# df_oxygen_A.head()
# df_oxygen_B.head()

# chemical data (ammonium, nitrite, phosphate) - recorded every minute
# df_ammonium.head()
# df_nitrite.head()
# df_phosphate.head()

# effluent data - recorded every 15 minutes
# df_effluent.head() 15min

# weather data - recorded every hour
# df_weather.head()

Define a function to find and ffill missing timestamps in two oxygen datasets.

In [44]:
def process_oxygen_data(df):
    """
    Cleans oxygen measurement data by:
    - Converting timestamps to datetime format
    - Removing duplicate timestamps (keeping the first occurrence)
    - Ensuring a continuous time index at 1-minute intervals
    - Forward filling missing 'hstWaarde' values

    Parameters:
        df (pd.DataFrame): Raw oxygen sensor data

    Returns:
        pd.DataFrame: Cleaned data with a datetime index
    """
    # Reset index if datumBeginMeting is already the index
    if 'datumBeginMeting' not in df.columns and df.index.name == 'datumBeginMeting':
        df = df.reset_index()

    # Step 1: Ensure the timestamp column is in datetime format
    df['datumBeginMeting'] = pd.to_datetime(df['datumBeginMeting'])

    # Step 2: Remove duplicate timestamps (keep the first occurrence)
    df = df.drop_duplicates(subset='datumBeginMeting', keep='first')

    # Step 3: Set the timestamp as the index
    df.set_index('datumBeginMeting', inplace=True)

    # Step 4: Generate a complete time range with 1-minute frequency
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='T')

    # Step 5: Reindex the DataFrame to include missing timestamps
    df = df.reindex(full_range)

    # Step 6: Report missing values
    missing_count = df['hstWaarde'].isna().sum()
    print(f"Number of missing timestamps: {missing_count}")
    if missing_count > 0:
        print("Example missing timestamps:", df[df['hstWaarde'].isna()].index[:5])

    # Step 7: Forward fill missing 'hstWaarde' values
    df.ffill(inplace=True)

    # Step 8: Set index name
    df.index.name = 'datumBeginMeting'

    return df

Apply the function to both oxygen dataframes and run twice to check if the missing valus filled.

In [45]:
df_oxygen_A = process_oxygen_data(df_oxygen_A)

Number of missing timestamps: 611
Example missing timestamps: DatetimeIndex(['2023-01-03 09:09:00', '2023-01-03 09:10:00',
               '2023-01-03 09:11:00', '2023-01-03 09:12:00',
               '2023-01-03 09:13:00'],
              dtype='datetime64[ns]', freq=None)


/var/folders/6m/nmdc67354c9f77107p22bnd00000gn/T/ipykernel_57222/442564830.py:29: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='T')


In [46]:
df_oxygen_A = process_oxygen_data(df_oxygen_A)

/var/folders/6m/nmdc67354c9f77107p22bnd00000gn/T/ipykernel_57222/442564830.py:29: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='T')


Number of missing timestamps: 0


In [47]:
df_oxygen_B = process_oxygen_data(df_oxygen_B)

/var/folders/6m/nmdc67354c9f77107p22bnd00000gn/T/ipykernel_57222/442564830.py:29: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='T')


Number of missing timestamps: 611
Example missing timestamps: DatetimeIndex(['2023-01-03 09:09:00', '2023-01-03 09:10:00',
               '2023-01-03 09:11:00', '2023-01-03 09:12:00',
               '2023-01-03 09:13:00'],
              dtype='datetime64[ns]', freq=None)


In [48]:
df_oxygen_B = process_oxygen_data(df_oxygen_B)

/var/folders/6m/nmdc67354c9f77107p22bnd00000gn/T/ipykernel_57222/442564830.py:29: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='T')


Number of missing timestamps: 0


Merge the oxygen datasets and take the average from them as the new column.

In [49]:
# reset index to get 'datumBeginMeting' back as a column and rename 'hstWaarde' columns to 'oxygen_a' and 'oxygen_b'
a_clean = df_oxygen_A.reset_index()[['datumBeginMeting', 'hstWaarde']].rename(columns={'hstWaarde': 'oxygen_a'})
b_clean = df_oxygen_B.reset_index()[['datumBeginMeting', 'hstWaarde']].rename(columns={'hstWaarde': 'oxygen_b'})

# merge two oxygen dataframes on 'datumBeginMeting'
df_oxygen = pd.merge(a_clean, b_clean, on='datumBeginMeting', how='inner')

# convert to numeric
df_oxygen['oxygen_a'] = pd.to_numeric(df_oxygen['oxygen_a'], errors='coerce')
df_oxygen['oxygen_b'] = pd.to_numeric(df_oxygen['oxygen_b'], errors='coerce')

# take the average of the two oxygen columns
df_oxygen['oxygen_avg'] = (df_oxygen['oxygen_a'] + df_oxygen['oxygen_b']) / 2

df_oxygen.head()

,datumBeginMeting,oxygen_a,oxygen_b,oxygen_avg
0,2023-01-01 00:00:00,1.663,0.777,1.2200
1,2023-01-01 00:01:00,1.789,0.982,1.3855
2,2023-01-01 00:02:00,2.105,1.189,1.6470
3,2023-01-01 00:03:00,2.271,1.266,1.7685
4,2023-01-01 00:04:00,2.168,1.403,1.7855


Define a function find and ffill missing timestamps in influent and chemicals datasets.

In [50]:
def fill_missing_timestamps(df, time_col='datumBeginMeting', freq='T', value_cols=None):
    """
    Ensure the time column is datetime, remove duplicates, fill missing timestamps with forward fill.
    Prints number of missing timestamps before and after filling.
    
    Parameters:
    - df: input dataframe
    - time_col: name of datetime column (default 'datumBeginMeting')
    - freq: frequency string for complete timestamp index (e.g., 'T' for minute)
    - value_cols: list of columns to keep (except time_col). If None, keeps all except time_col.
    
    Returns:
    - processed dataframe with no missing timestamps, index is time_col
    """
    df = df.copy()
    
    # Convert time column to datetime
    df[time_col] = pd.to_datetime(df[time_col])
    
    # Remove duplicates in time_col, keep first
    df = df.drop_duplicates(subset=time_col, keep='first')
    
    # Set datetime as index
    df.set_index(time_col, inplace=True)
    
    # Keep only specified columns if provided
    if value_cols is not None:
        df = df[value_cols]
    
    # Calculate full timestamp range with given frequency
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)
    
    # Calculate missing timestamps count before reindexing
    missing_before = len(full_range) - len(df)
    print(f"Missing timestamps before filling: {missing_before}")
    
    # Reindex to include missing timestamps
    df = df.reindex(full_range)
    
    # Fill missing values forward
    df.ffill(inplace=True)
    
    # Calculate missing timestamps count after filling
    missing_after = df.isna().sum().sum()
    print(f"Missing timestamps after filling: {missing_after}")
    
    # Rename index back to time_col
    df.index.name = time_col
    
    return df

In [51]:
# Process influent data
df_influent_filled = fill_missing_timestamps(df_influent, time_col='datumBeginMeting', freq='T', value_cols=['hstWaarde'])

# Process oxygen data
df_oxygen_filled = fill_missing_timestamps(df_oxygen, time_col='datumBeginMeting', freq='T', value_cols=['oxygen_avg'])

# Process ammonium data
df_ammonium_filled = fill_missing_timestamps(df_ammonium, time_col='datumBeginMeting', freq='T', value_cols=['hstWaarde'])

# Process nitrite data
df_nitrate_filled = fill_missing_timestamps(df_nitrite, time_col='datumBeginMeting', freq='T', value_cols=['hstWaarde'])

# Process phosphate data
df_phosphate_filled = fill_missing_timestamps(df_phosphate, time_col='datumBeginMeting', freq='T', value_cols=['hstWaarde'])

/var/folders/6m/nmdc67354c9f77107p22bnd00000gn/T/ipykernel_57222/1828636121.py:31: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)
/var/folders/6m/nmdc67354c9f77107p22bnd00000gn/T/ipykernel_57222/1828636121.py:31: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)
/var/folders/6m/nmdc67354c9f77107p22bnd00000gn/T/ipykernel_57222/1828636121.py:31: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)


Missing timestamps before filling: 421
Missing timestamps after filling: 0
Missing timestamps before filling: 0
Missing timestamps after filling: 0
Missing timestamps before filling: 611
Missing timestamps after filling: 0
Missing timestamps before filling: 611
Missing timestamps after filling: 0
Missing timestamps before filling: 611
Missing timestamps after filling: 0


/var/folders/6m/nmdc67354c9f77107p22bnd00000gn/T/ipykernel_57222/1828636121.py:31: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)
/var/folders/6m/nmdc67354c9f77107p22bnd00000gn/T/ipykernel_57222/1828636121.py:31: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)


From the results above, we can see that Oxygen A, Oxygen B, and all the chemical datasets each had 611 missing timestamps, while Influent had 421 missing timestamps.
These missing values were filled using forward fill (ffill) since the data are time series.

In [52]:
# Prepare for merging — reset index and rename columns
influent_clean = df_influent_filled.reset_index().rename(columns={'hstWaarde': 'Influent'})
oxygen_clean = df_oxygen_filled.reset_index().rename(columns={'hstWaarde': 'Oxygen'})
ammonium_clean = df_ammonium_filled.reset_index().rename(columns={'hstWaarde': 'Ammonium'})
nitrate_clean = df_nitrate_filled.reset_index().rename(columns={'hstWaarde': 'Nitrate'})
phosphate_clean = df_phosphate_filled.reset_index().rename(columns={'hstWaarde': 'Phosphate'})

# Merge all datasets on the timestamp column
df_water = influent_clean.merge(oxygen_clean, on='datumBeginMeting', how='inner') \
                        .merge(ammonium_clean, on='datumBeginMeting', how='inner') \
                        .merge(nitrate_clean, on='datumBeginMeting', how='inner') \
                        .merge(phosphate_clean, on='datumBeginMeting', how='inner')

# Convert datumBeginMeting to datetime and set as index
df_water['datumBeginMeting'] = pd.to_datetime(df_water['datumBeginMeting'])
df_water.set_index('datumBeginMeting', inplace=True)

# Convert all columns to numeric, coercing errors to NaN
df_water = df_water.apply(pd.to_numeric, errors='coerce')

# Resample hourly and take the mean
df_water = df_water.resample('H').mean().reset_index()

df_water.head()

/var/folders/6m/nmdc67354c9f77107p22bnd00000gn/T/ipykernel_57222/3555450314.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_water = df_water.resample('H').mean().reset_index()


,datumBeginMeting,Influent,oxygen_avg,Ammonium,Nitrate,Phosphate
0,2023-01-01 00:00:00,3202.204867,0.919150,1.037983,4.424633,0.003450
1,2023-01-01 01:00:00,2790.287717,0.895517,1.938650,3.062367,0.380733
2,2023-01-01 02:00:00,2281.950900,1.113550,1.885600,3.344783,0.169217
3,2023-01-01 03:00:00,2377.076550,0.584708,0.933100,2.867483,0.161217
4,2023-01-01 04:00:00,1517.581467,0.552192,0.772933,2.491967,0.112783


After reviewing the weather dataset, I found that the Rain column only contains binary values: 0 indicates no rain, and 1 indicates rain. So, I decided to use the PrecipitationAmount column this time, which records the actual hourly precipitation amount in millimeters (mm). Additionally, the temperature values in the dataset have already been adjusted by dividing them by 10, so the unit is now degrees Celsius (°C).

In [53]:
# Convert datetime columns to datetime type
df_weather['Timestamp'] = pd.to_datetime(df_weather['Timestamp'])
df_water['datumBeginMeting'] = pd.to_datetime(df_water['datumBeginMeting'])

# Rename datetime columns to 'date'
df_weather = df_weather.rename(columns={'Timestamp': 'date', 'PrecipitationAmount': 'Rainfall'})
df_water = df_water.rename(columns={'datumBeginMeting': 'date'})

# Merge the two dataframes on 'date'
df_main = pd.merge(df_water, df_weather, on='date', how='inner')

df_main.to_csv("data/main_2023.csv", index=False)

# Display the first few rows of the merged dataframe
df_main.head()


,date,Influent,oxygen_avg,Ammonium,Nitrate,Phosphate,Rainfall,Temperature
0,2023-01-01 00:00:00,3202.204867,0.919150,1.037983,4.424633,0.003450,0,15.3
1,2023-01-01 01:00:00,2790.287717,0.895517,1.938650,3.062367,0.380733,0,14.6
2,2023-01-01 02:00:00,2281.950900,1.113550,1.885600,3.344783,0.169217,0,15.0
3,2023-01-01 03:00:00,2377.076550,0.584708,0.933100,2.867483,0.161217,0,14.7
4,2023-01-01 04:00:00,1517.581467,0.552192,0.772933,2.491967,0.112783,0,14.1


Apply min-max scaling to the main dataset.

In [54]:
from sklearn.preprocessing import MinMaxScaler

# Separate the 'date' column
df_features = df_main.drop(columns=['date'])

# Initialize the scaler
scaler = MinMaxScaler()

# Apply min-max scaling to all feature columns
df_scaled_values = scaler.fit_transform(df_features)

# Create a new DataFrame with scaled values and the same column names
df_scaled = pd.DataFrame(df_scaled_values, columns=df_features.columns)

# Add the 'date' column back
df_scaled['date'] = df_main['date']

# Reorder columns to place 'date' first
df_scaled = df_scaled[['date'] + df_features.columns.tolist()]

df_scaled.to_csv("data/scaled_main_2023.csv", index=False)

# Display the first few rows of the scaled DataFrame
df_scaled.head()


,date,Influent,oxygen_avg,Ammonium,Nitrate,Phosphate,Rainfall,Temperature
0,2023-01-01 00:00:00,0.441427,0.306547,0.051943,0.420280,0.000095,0.0,0.556391
1,2023-01-01 01:00:00,0.381769,0.298638,0.097014,0.278328,0.063068,0.0,0.538847
2,2023-01-01 02:00:00,0.308147,0.371606,0.094359,0.307757,0.027763,0.0,0.548872
3,2023-01-01 03:00:00,0.321924,0.194623,0.046694,0.258021,0.026428,0.0,0.541353
4,2023-01-01 04:00:00,0.197444,0.183740,0.038679,0.218891,0.018344,0.0,0.526316
